# Exercise 1

In this exercise you do not have to fill out missing TODOs. Now, we want you to **create** and **train** a neural network for the EMNIST dataset[1] (balanced version), an extension of the MNIST dataset.

**Scoring System**: According to the following table, you will get points based on the **final test accuracy** of your model.

| Test Acc | Points |
| -------: | :----: |
| <= 84%   |   1    |
|    85%   |   2    |
|    86%   |   3    |
| >= 87%   |   4    |


Here are some notes which might help you:
* Please train your model not just once. We will use your code to retrain a model multiple times so you have to avoid lucky shots.
* The preprocessing of the data can be realized with different transformations[2]
* When changing the last activation function an appropriate loss function has to be selected and vice versa
  (The given model uses the LogSoftmax in combination with the negative log likelihood loss)
* There are two typical structures for a neural network:
    * Only linear layers: Each linear layer is followed by an activation function.
    * Convolutional layers followed by a few linear layers: ConvLayers each with its own activation function followed by a pooling function (standard: MaxPool2D). Afterwards, a few linear layers with activation functions are added.
* You can also use convolutional layers[3] which are especially useful for image classification / segmentation tasks.

[1] https://www.nist.gov/itl/products-and-services/emnist-dataset<br>
[2] https://pytorch.org/vision/stable/transforms.html<br>
[3] https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from matplotlib import pyplot as plt
import string

## Create and preprocess EMNIST

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),])

train_dataset = datasets.EMNIST("./data", split="balanced", train=True, download=True, transform=transform)
test_dataset = datasets.EMNIST("./data", split="balanced", train=False, download=True, transform=transform)

## Explore the data points

In [ ]:
class_names = list(string.digits + string.ascii_uppercase) + ["a", "b", "d", "e", "f", "g", "h", "n", "q", "r", "t"]

figure = plt.figure(figsize=(10, 8))
cols, rows = 5, 5
sample_idx = torch.randint(0, 112800, size=(25,))

for i in range(1, cols * rows + 1):
    img, label = train_dataset.data[sample_idx[i-1]],  train_dataset.targets[sample_idx[i-1]]
    figure.add_subplot(rows, cols, i)
    plt.title(class_names[label])
    plt.axis("off")
    plt.imshow(torch.transpose(img.squeeze(), 1, 0), cmap="gray")
plt.show()

## Create a neural network

In [ ]:
class Net(nn.Module):
    
    ############
    #   TODO   #
    ############
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # <- 28*28 = 784
        self.fc2 = nn.Linear(128, 47)

    def forward(self, x):
        x = torch.flatten(x, 1)  # <- Linear layers only process 2d data with shape (batch_size, data_size)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)  # <- Works with <nll_loss> (negative log likelihood loss)
        return output

## Train and test

In [ ]:
def train(model, device, train_loader, optimizer):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
    print("Train Loss: {:.6f}".format(loss.item()))



def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)

    print("Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

## Main

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 32}
test_kwargs = {'batch_size': 1000}
    
if use_cuda:
    cuda_kwargs = {'num_workers': 2,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)
    
train_loader = torch.utils.data.DataLoader(train_dataset,**train_kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters())

for epoch in range(1, 10):
    print(f"Epoch {epoch}")
    train(model, device, train_loader, optimizer)
    test(model, device, test_loader)